# Testing Notebook 02:

In [1]:
!pip install ..

from typing import List
import pandas as pd 
import numpy as np

from statdepth.depth import banddepth
from statdepth.depth.depth import _univariate_band_depth, _handle_depth_errors
from statdepth.depth._containment import _select_containment



Processing /Users/julian/Documents/Projects/statdepth
  Created wheel for statdepth: filename=statdepth-0.1.0-py2.py3-none-any.whl size=10809 sha256=8804a42c2dda8d5b3c7df4e2789ddda445009f806331eab7e2e2ff54e2477cd3
  Stored in directory: /private/var/folders/pd/jsjcl0fn7w57s5mfr34b20pm0000gn/T/pip-ephem-wheel-cache-evgwxh6f/wheels/94/78/65/e7cb177a6619a78b187853c88271a8abc224aa673b05e4ed56
Successfully built statdepth
  Attempting uninstall: statdepth
    Found existing installation: statdepth 0.1.0
    Uninstalling statdepth-0.1.0:
      Successfully uninstalled statdepth-0.1.0


In [2]:
!where python

/Users/julian/opt/miniconda3/bin/python
/usr/bin/python


Generate a toy DataFrame for testing

In [3]:
t = [1,2,3,4,4,4,4.5,2,3,2,3]*2
df = pd.DataFrame()
for col in range(10):
    df[col] = t
    
for col in df:
    df[col] = df[col] * np.random.rand() * 2

df

,0,1,2,3,4,5,6,7,8,9
0,0.671904,0.960409,1.352264,1.221487,1.003199,1.998476,0.261355,1.684903,0.138637,0.724736
1,1.343809,1.920818,2.704528,2.442974,2.006399,3.996952,0.522711,3.369806,0.277274,1.449472
2,2.015713,2.881226,4.056792,3.664461,3.009598,5.995428,0.784066,5.054709,0.415910,2.174208
3,2.687618,3.841635,5.409057,4.885949,4.012798,7.993903,1.045422,6.739612,0.554547,2.898944
4,2.687618,3.841635,5.409057,4.885949,4.012798,7.993903,1.045422,6.739612,0.554547,2.898944
5,2.687618,3.841635,5.409057,4.885949,4.012798,7.993903,1.045422,6.739612,0.554547,2.898944
6,3.023570,4.321839,6.085189,5.496692,4.514398,8.993141,1.176099,7.582063,0.623866,3.261312
7,1.343809,1.920818,2.704528,2.442974,2.006399,3.996952,0.522711,3.369806,0.277274,1.449472
8,2.015713,2.881226,4.056792,3.664461,3.009598,5.995428,0.784066,5.054709,0.415910,2.174208
9,1.343809,1.920818,2.704528,2.442974,2.006399,3.996952,0.522711,3.369806,0.277274,1.449472


In [45]:
def samplebanddepth(data: List[pd.DataFrame], K: int, J=2, containment='r2', relax=False, deep_check=False):
    samples = []

    # Handle common errros
    _handle_depth_errors(data=data, J=J, containment=containment, relax=relax, deep_check=deep_check)

    cdef = _select_containment(containment=containment)
    
    # Univariate case
    if len(data) == 1:
        df = data[0]
        orig = df.copy()
        ss = df.shape[1] // K
        
        for col in orig.columns:
            depths = []

            for i in range(K):
                t = df.sample(n=ss, axis=1)
                df = df.drop(t.columns, axis=1)
                depths.append(_univariate_band_depth(data=t, curve=orig[col], relax=relax, containment=cdef, J=J))
            
            samples.append(depths)
            df = orig.copy()
        samples = pd.Series(index=df.columns, data=[np.mean(i) for i in samples])
    else:
        # Multivariate case: partition list of DataFrames randomly, compute band depth w.r.t those
        shuffled = data.copy()
        random.shuffle(shuffled)
        ss = len(data) // K 

        for _ in range(K):
            pass
        
    return samples

In [52]:
%%timeit
samplebanddepth([df], K=3)

0    0.666667
1    0.555556
2    0.222222
3    0.777778
4    0.444444
5    0.222222
6    0.333333
7    0.222222
8    0.000000
9    0.777778
dtype: float64

In [50]:
%%timeit
banddepth([df])

303 ms ± 9.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
df.sample(n=5, axis=1)

,2,9,5,4,0
0,1.352264,0.724736,1.998476,1.003199,0.671904
1,2.704528,1.449472,3.996952,2.006399,1.343809
2,4.056792,2.174208,5.995428,3.009598,2.015713
3,5.409057,2.898944,7.993903,4.012798,2.687618
4,5.409057,2.898944,7.993903,4.012798,2.687618
5,5.409057,2.898944,7.993903,4.012798,2.687618
6,6.085189,3.261312,8.993141,4.514398,3.023570
7,2.704528,1.449472,3.996952,2.006399,1.343809
8,4.056792,2.174208,5.995428,3.009598,2.015713
9,2.704528,1.449472,3.996952,2.006399,1.343809
